In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import math

In [2]:
ticker = 'High_Value_Dataset_July_2021'
input_dir = r'./data/'
file = os.path.join(input_dir, ticker + '.csv')

df = pd.read_csv(file)
df.head(5)

,SID Number,TDCJ Number,Name,Current Facility,Gender,Race,Age,Projected Release,Maximum Sentence Date,Parole Eligibility Date,Case Number,County,Offense Code,TDCJ Offense,Sentence Date,Offense Date,Sentence (Years),Last Parole Decision,Next Parole Review Date,Parole Review Status
0,524524,758486,"HARPER,DAVID JOHN JR",Estelle,M,W,92,01/01/9999,01/01/9999,08/03/2026,95-765-C,McLennan,11180000,AGG SEXUAL ASSAULT,08/13/1996,09/15/1995,Life,None,08/03/2026,NOT IN REVIEW PROCESS
1,703267,841623,"RODRIGUEZ,THEODORE",Beto,M,W,83,08/03/2043,08/03/2043,02/01/2021,97-396-C,McLennan,11990003,AGG SEX ASLT,08/05/1998,06/01/1996,45.0,None,Unavailable at this time.,IN PAROLE REVIEW PROCESS
2,766785,365547,"BROWN,NARRIES EARL",W. Scott,M,W,85,01/01/9999,01/01/9999,08/24/2003,F83-89728-HI,Dallas,11220000,AGG SEX ABUSE CHILD/U14,08/24/1983,03/15/1983,Life,Denied on 12/05/2016,Unavailable at this time.,NOT IN REVIEW PROCESS
3,770626,449674,"FLORES,ISABEL",Pack,M,H,84,01/01/9999,01/01/9999,09/02/2006,86CR-1234-B,Nueces,9150000,MURDER W/DEADLY WPN,04/13/1987,09/02/1986,Life,Denied on 07/21/2021,07/2024,NOT IN REVIEW PROCESS
4,771601,1491019,"MOLETT,JOHN HENRY",Pack,M,B,79,12/02/2037,12/02/2037,12/02/2022,1144294,Harris,12990002,AGG ROBBERY W/DW,02/21/2008,12/03/2007,30.0,None,12/02/2022,NOT IN REVIEW PROCESS


In [3]:
df.isna().sum()

SID Number                     0
TDCJ Number                    0
Name                           0
Current Facility               0
Gender                         0
Race                           0
Age                            0
Projected Release             11
Maximum Sentence Date       1914
Parole Eligibility Date     9847
Case Number                   38
County                         0
Offense Code                   0
TDCJ Offense                   0
Sentence Date               1990
Offense Date                   0
Sentence (Years)              31
Last Parole Decision        8857
Next Parole Review Date    22368
Parole Review Status        8857
dtype: int64

In [4]:
# drop next parole review date because those who were approved do not have one
df = df.drop(columns='Next Parole Review Date')

# drop all rows with null values, drop where doesn't have a last parole decision
df = df.dropna()
df = df[df['Last Parole Decision'] != 'None']


# drop columns because they are not useful
drop_cols = ['Name', 'SID Number', 'Case Number']
df = df.drop(columns=drop_cols)

In [5]:
def encode_and_bind(original_dataframe, feature_to_encode):
    dummies = pd.get_dummies(original_dataframe[[feature_to_encode]])
    res = pd.concat([original_dataframe, dummies], axis=1)
    res = res.drop([feature_to_encode], axis=1)
    return(res) 

In [6]:
## Change data types here
le = LabelEncoder()
# Categorical to numerical
df['Gender'] = le.fit_transform(df['Gender'])
df['Race'] = le.fit_transform(df['Race'])
df['TDCJ Offense'] = le.fit_transform(df['TDCJ Offense'])
df['County'] = le.fit_transform(df['County'])
df['Current Facility'] = le.fit_transform(df['Current Facility'])
df['Age'].astype(int)


df['Last Parole Decision'] = np.where(df['Last Parole Decision'].str.startswith('Approve'), 1, 0)

## If Parole Review Status is IN PAROLE REVIEW PROCESS, then 1, else 0
df['Parole Review Status'] = np.where(df['Parole Review Status'] == 'IN PAROLE REVIEW PROCESS', 1, 0)

df['Sentence (Years)'] = np.where((df['Sentence (Years)'] == 'Capital Life') | (df['Sentence (Years)'] == 'Life'), 999, df['Sentence (Years)'])
df['Sentence (Years)'] = df['Sentence (Years)'].astype(float)

## converting date columns to date columns
df['Sentence Date'] = pd.to_datetime(df['Sentence Date'], format='%m/%d/%Y')
df['Offense Date'] = pd.to_datetime(df['Offense Date'], format='%m/%d/%Y')

In [7]:
df.head()

,TDCJ Number,Current Facility,Gender,Race,Age,Projected Release,Maximum Sentence Date,Parole Eligibility Date,County,Offense Code,TDCJ Offense,Sentence Date,Offense Date,Sentence (Years),Last Parole Decision,Parole Review Status
2,365547,92,1,6,85,01/01/9999,01/01/9999,08/24/2003,55,11220000,1087,1983-08-24,1983-03-15,999.0,0,0
3,449674,71,1,2,84,01/01/9999,01/01/9999,09/02/2006,173,9150000,7178,1987-04-13,1986-09-02,999.0,0,0
5,1241039,9,1,1,83,06/19/2023,06/19/2023,06/18/2013,204,11990003,1380,2004-05-13,1999-07-24,20.0,0,0
6,248098,86,1,6,84,01/01/9999,01/01/9999,05/24/1981,99,9130000,3487,1975-04-30,1974-06-28,999.0,0,1
8,1191104,26,1,2,82,04/15/2023,04/15/2023,06/15/2005,218,36120005,4952,2003-08-15,2003-04-08,20.0,0,0


In [8]:
# find the current date to set all other dates against (number of days away)
current_date = 0

if max(df['Sentence Date']) > max(df['Offense Date']):
    current_date = max(df['Sentence Date'])
else:
    current_date = max(df['Offense Date'])
print(current_date)

2021-07-12 00:00:00


In [9]:
# find number of days Parole Eligibility is away from current date
pe_date = []

for i in df['Parole Eligibility Date']:
    if i != '01/01/9999':
        temp = pd.to_datetime(i, format='%m/%d/%Y')
        num_years = temp - current_date
        num_years = round((num_years / np.timedelta64(1, 'Y')))
        pe_date.append(num_years)

df['Parole Eligibility (Years)'] = pe_date

In [10]:
# find number of days Max Release Date  is away from current date
ms_date = []

## Simplified
for i in df['Maximum Sentence Date']:
    """
    If the sentence is more than current_date + 100 years, then set it to 999 Years
    """
    if int(i[-4:]) > current_date.year + 100:
        ms_date.append(999)
    else:
        temp = pd.to_datetime(i, format='%m/%d/%Y')
        num_years = temp - current_date
        num_years = round((num_years/np.timedelta64(1, 'Y')))
        ms_date.append(num_years)

df['Sentence Left (Years)'] = ms_date

In [11]:
df.head(5)

,TDCJ Number,Current Facility,Gender,Race,Age,Projected Release,Maximum Sentence Date,Parole Eligibility Date,County,Offense Code,TDCJ Offense,Sentence Date,Offense Date,Sentence (Years),Last Parole Decision,Parole Review Status,Parole Eligibility (Years),Sentence Left (Years)
2,365547,92,1,6,85,01/01/9999,01/01/9999,08/24/2003,55,11220000,1087,1983-08-24,1983-03-15,999.0,0,0,-18,999
3,449674,71,1,2,84,01/01/9999,01/01/9999,09/02/2006,173,9150000,7178,1987-04-13,1986-09-02,999.0,0,0,-15,999
5,1241039,9,1,1,83,06/19/2023,06/19/2023,06/18/2013,204,11990003,1380,2004-05-13,1999-07-24,20.0,0,0,-8,2
6,248098,86,1,6,84,01/01/9999,01/01/9999,05/24/1981,99,9130000,3487,1975-04-30,1974-06-28,999.0,0,1,-40,999
8,1191104,26,1,2,82,04/15/2023,04/15/2023,06/15/2005,218,36120005,4952,2003-08-15,2003-04-08,20.0,0,0,-16,2


In [12]:
df['Parole Eligibility (Years)'] = np.abs(df['Parole Eligibility (Years)'])

Final Preprocessed Columns

In [13]:
columns = ['Gender', 'Race', 'Age', 'County', 'Current Facility', 'Offense Code', 'TDCJ Offense', 'TDCJ Number', 'Sentence (Years)', 'Last Parole Decision', 'Parole Review Status', 'Parole Eligibility (Years)', 'Sentence Left (Years)']
df = df[columns]
df.head(5)

,Gender,Race,Age,County,Current Facility,Offense Code,TDCJ Offense,TDCJ Number,Sentence (Years),Last Parole Decision,Parole Review Status,Parole Eligibility (Years),Sentence Left (Years)
2,1,6,85,55,92,11220000,1087,365547,999.0,0,0,18,999
3,1,2,84,173,71,9150000,7178,449674,999.0,0,0,15,999
5,1,1,83,204,9,11990003,1380,1241039,20.0,0,0,8,2
6,1,6,84,99,86,9130000,3487,248098,999.0,0,1,40,999
8,1,2,82,218,26,36120005,4952,1191104,20.0,0,0,16,2


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61209 entries, 2 to 118317
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Gender                      61209 non-null  int32  
 1   Race                        61209 non-null  int32  
 2   Age                         61209 non-null  int64  
 3   County                      61209 non-null  int32  
 4   Current Facility            61209 non-null  int32  
 5   Offense Code                61209 non-null  int64  
 6   TDCJ Offense                61209 non-null  int32  
 7   TDCJ Number                 61209 non-null  int64  
 8   Sentence (Years)            61209 non-null  float64
 9   Last Parole Decision        61209 non-null  int32  
 10  Parole Review Status        61209 non-null  int32  
 11  Parole Eligibility (Years)  61209 non-null  int64  
 12  Sentence Left (Years)       61209 non-null  int64  
dtypes: float64(1), int32(7), int64

#### 

In [43]:
df.to_csv(r'./data/' + ticker + '_preprocessed.csv')